In [14]:
import tensorflow as tf
import numpy as np
import cv2
from keras.utils import Sequence
from keras.applications.mobilenet_v3 import preprocess_input
from glob import glob
import os
from sklearn.model_selection import train_test_split

from keras.applications import MobileNetV3Large
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import TopKCategoricalAccuracy
from keras.layers import RandomRotation
from keras.callbacks import LearningRateScheduler



import math
import json


import re
print(tf.__version__)
import tensorflow_probability as tfp
tfd = tfp.distributions

2.15.0


In [ ]:
# Load class names
with open("food-101/meta/classes.txt", "r") as file:
    classes = [line.strip() for line in file]

# Read image paths
with open("food-101/meta/train.txt", "r") as file:
    train_paths = [line.strip() for line in file]

with open("food-101/meta/test.txt", "r") as file:
    test_paths = [line.strip() for line in file]

# Helper function to convert label format
def to_snake_case(name):
    return name.replace(" ", "_").lower()

# Convert paths to full paths
train_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in train_paths]
train_labels = [to_snake_case(p.split("/")[0]) for p in train_paths]

test_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in test_paths]
test_labels = [to_snake_case(p.split("/")[0]) for p in test_paths]

# Map labels to indices
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
train_label_indices = [class_to_index[label] for label in train_labels]
test_label_indices = [class_to_index[label] for label in test_labels]

# ✂️ Split 80% train, 20% val
train_paths_split, val_paths_split, train_labels_split, val_labels_split = train_test_split(
    train_full_paths, train_label_indices, test_size=0.2, stratify=train_label_indices, random_state=42
)

# Constants
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = len(classes)

rotation_layer = RandomRotation(factor=0.014)

# Augmentation function
def augment_image(img):
    img = rotation_layer(img)
    img = tf.image.resize_with_crop_or_pad(img, 256, 256)
    img = tf.image.random_crop(img, size=[224, 224, 3])    
    img = tf.image.random_flip_left_right(img)            
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.image.random_saturation(img, 0.9, 1.1)
    img = tf.image.random_hue(img, 0.08)
    img = tf.clip_by_value(img, 0.0, 255.0)
    return img

# Preprocess each sample
def process_example(path, label, augment=False):
    img_raw = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    if augment:
        img = augment_image(img)
    img = preprocess_input(img)
    label = tf.one_hot(label, NUM_CLASSES)
    return img, label

# Dataset creation function
def create_dataset(paths, labels, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.map(lambda x, y: process_example(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

val_ds = create_dataset(val_paths_split, val_labels_split, augment=False, shuffle=False)
test_ds = create_dataset(test_full_paths, test_label_indices, shuffle=False)

In [17]:
def get_cosine_annealing_with_restarts(
    initial_lr=1e-3,
    min_lr=1e-7,
    warmup_epochs=5,
    T_0=20,
    T_mult=1
):
    def scheduler(epoch, lr):

        # Warmup phase
        if epoch < warmup_epochs:
            return float(initial_lr * (epoch + 1) / warmup_epochs)

        # Cosine annealing with restarts
        t = epoch - warmup_epochs  # Epochs after warmup
        T_i = T_0
        cumulative = 0

        while t >= T_i:
            t -= T_i
            cumulative += T_i
            T_i *= T_mult

        progress = t / T_i
        cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))
        return float(min_lr + (initial_lr - min_lr) * cosine_decay)

    return scheduler

In [18]:
# No Augmentation
train_ds = create_dataset(train_paths_split, train_labels_split, augment=False)

loss_fn = CategoricalCrossentropy(label_smoothing=0.05)


base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu')(x)
output = Dense(101, activation='softmax')(x)

checkpoint_cb = ModelCheckpoint(
    filepath='D:\\Skripsi\\MobilNetV3-Food101-Rev-Baseline\\Baseline_Model_MobileNetV3.keras',          
    monitor='val_accuracy',            
    mode='max',                         
    save_best_only=True,                            
    verbose=1
)

lr_scheduler = LearningRateScheduler(get_cosine_annealing_with_restarts())

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5, 
    callbacks=[checkpoint_cb, lr_scheduler]
)

total_layers = len(base_model.layers)

unfreeze_count = int(total_layers * 0.3)

for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history_fine_tune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=45,
    initial_epoch=5,
    callbacks=[checkpoint_cb, lr_scheduler]
)

combined_history = {}
for key in history.history.keys():
    combined_history[key] = history.history[key] + history_fine_tune.history[key]

with open('skripsi/history/Baseline_History_MobilNetV3.json', 'w') as f:
    json.dump(combined_history, f)

Epoch 1/5
947/947 [==============================] - ETA: 0s - loss: 3.1558 - accuracy: 0.3063 - top_5_accuracy: 0.5474
Epoch 1: val_accuracy improved from -inf to 0.51630, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 708s 746ms/step - loss: 3.1558 - accuracy: 0.3063 - top_5_accuracy: 0.5474 - val_loss: 2.2249 - val_accuracy: 0.5163 - val_top_5_accuracy: 0.7806 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 2.2506 - accuracy: 0.5032 - top_5_accuracy: 0.7752
Epoch 2: val_accuracy improved from 0.51630 to 0.58337, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 666s 704ms/step - loss: 2.2506 - accuracy: 0.5032 - top_5_accuracy: 0.7752 - val_loss: 1.9664 - val_accuracy: 0.5834 - val_top_5_accuracy: 0.8309 - lr: 4.0000e-04
Epoch 3/5
947/947 [==============================] - ET

TypeError: Object of type float32 is not JSON serializable

Epoch 1/5
947/947 [==============================] - ETA: 0s - loss: 3.1558 - accuracy: 0.3063 - top_5_accuracy: 0.5474
Epoch 1: val_accuracy improved from -inf to 0.51630, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 708s 746ms/step - loss: 3.1558 - accuracy: 0.3063 - top_5_accuracy: 0.5474 - val_loss: 2.2249 - val_accuracy: 0.5163 - val_top_5_accuracy: 0.7806 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 2.2506 - accuracy: 0.5032 - top_5_accuracy: 0.7752
Epoch 2: val_accuracy improved from 0.51630 to 0.58337, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 666s 704ms/step - loss: 2.2506 - accuracy: 0.5032 - top_5_accuracy: 0.7752 - val_loss: 1.9664 - val_accuracy: 0.5834 - val_top_5_accuracy: 0.8309 - lr: 4.0000e-04
Epoch 3/5
947/947 [==============================] - ETA: 0s - loss: 2.0445 - accuracy: 0.5563 - top_5_accuracy: 0.8160
Epoch 3: val_accuracy improved from 0.58337 to 0.59875, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 672s 709ms/step - loss: 2.0445 - accuracy: 0.5563 - top_5_accuracy: 0.8160 - val_loss: 1.8865 - val_accuracy: 0.5987 - val_top_5_accuracy: 0.8461 - lr: 6.0000e-04
Epoch 4/5
947/947 [==============================] - ETA: 0s - loss: 1.9500 - accuracy: 0.5815 - top_5_accuracy: 0.8335
Epoch 4: val_accuracy improved from 0.59875 to 0.61267, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 660s 697ms/step - loss: 1.9500 - accuracy: 0.5815 - top_5_accuracy: 0.8335 - val_loss: 1.8443 - val_accuracy: 0.6127 - val_top_5_accuracy: 0.8535 - lr: 8.0000e-04
Epoch 5/5
947/947 [==============================] - ETA: 0s - loss: 1.9058 - accuracy: 0.5920 - top_5_accuracy: 0.8424
Epoch 5: val_accuracy did not improve from 0.61267
947/947 [==============================] - 660s 697ms/step - loss: 1.9058 - accuracy: 0.5920 - top_5_accuracy: 0.8424 - val_loss: 1.8619 - val_accuracy: 0.6073 - val_top_5_accuracy: 0.8433 - lr: 0.0010
Epoch 6/45
947/947 [==============================] - ETA: 0s - loss: 1.8012 - accuracy: 0.6250 - top_5_accuracy: 0.8606
Epoch 6: val_accuracy did not improve from 0.61267
947/947 [==============================] - 897s 943ms/step - loss: 1.8012 - accuracy: 0.6250 - top_5_accuracy: 0.8606 - val_loss: 5.2479 - val_accuracy: 0.2968 - val_top_5_accuracy: 0.5476 - lr: 0.0010
Epoch 7/45
947/947 [==============================] - ETA: 0s - loss: 1.4224 - accuracy: 0.7303 - top_5_accuracy: 0.9182
Epoch 7: val_accuracy did not improve from 0.61267
947/947 [==============================] - 893s 943ms/step - loss: 1.4224 - accuracy: 0.7303 - top_5_accuracy: 0.9182 - val_loss: 2.0891 - val_accuracy: 0.5917 - val_top_5_accuracy: 0.8321 - lr: 9.9384e-04
Epoch 8/45
947/947 [==============================] - ETA: 0s - loss: 1.2304 - accuracy: 0.7851 - top_5_accuracy: 0.9437
Epoch 8: val_accuracy improved from 0.61267 to 0.61492, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 895s 945ms/step - loss: 1.2304 - accuracy: 0.7851 - top_5_accuracy: 0.9437 - val_loss: 1.9688 - val_accuracy: 0.6149 - val_top_5_accuracy: 0.8488 - lr: 9.7553e-04
Epoch 9/45
947/947 [==============================] - ETA: 0s - loss: 1.0824 - accuracy: 0.8279 - top_5_accuracy: 0.9629
Epoch 9: val_accuracy improved from 0.61492 to 0.68376, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 893s 943ms/step - loss: 1.0824 - accuracy: 0.8279 - top_5_accuracy: 0.9629 - val_loss: 1.6565 - val_accuracy: 0.6838 - val_top_5_accuracy: 0.8872 - lr: 9.4551e-04
Epoch 10/45
947/947 [==============================] - ETA: 0s - loss: 0.9494 - accuracy: 0.8685 - top_5_accuracy: 0.9760
Epoch 10: val_accuracy improved from 0.68376 to 0.68865, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 894s 944ms/step - loss: 0.9494 - accuracy: 0.8685 - top_5_accuracy: 0.9760 - val_loss: 1.6606 - val_accuracy: 0.6886 - val_top_5_accuracy: 0.8856 - lr: 9.0452e-04
Epoch 11/45
947/947 [==============================] - ETA: 0s - loss: 0.8449 - accuracy: 0.8993 - top_5_accuracy: 0.9861
Epoch 11: val_accuracy improved from 0.68865 to 0.69894, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 894s 945ms/step - loss: 0.8449 - accuracy: 0.8993 - top_5_accuracy: 0.9861 - val_loss: 1.6029 - val_accuracy: 0.6989 - val_top_5_accuracy: 0.8867 - lr: 8.5357e-04
Epoch 12/45
947/947 [==============================] - ETA: 0s - loss: 0.7502 - accuracy: 0.9301 - top_5_accuracy: 0.9927
Epoch 12: val_accuracy improved from 0.69894 to 0.71993, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 893s 943ms/step - loss: 0.7502 - accuracy: 0.9301 - top_5_accuracy: 0.9927 - val_loss: 1.5445 - val_accuracy: 0.7199 - val_top_5_accuracy: 0.8933 - lr: 7.9391e-04
Epoch 13/45
947/947 [==============================] - ETA: 0s - loss: 0.6857 - accuracy: 0.9503 - top_5_accuracy: 0.9956
Epoch 13: val_accuracy did not improve from 0.71993
947/947 [==============================] - 893s 943ms/step - loss: 0.6857 - accuracy: 0.9503 - top_5_accuracy: 0.9956 - val_loss: 1.5426 - val_accuracy: 0.7151 - val_top_5_accuracy: 0.8967 - lr: 7.2702e-04
Epoch 14/45
947/947 [==============================] - ETA: 0s - loss: 0.6265 - accuracy: 0.9683 - top_5_accuracy: 0.9982
Epoch 14: val_accuracy improved from 0.71993 to 0.73003, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 893s 943ms/step - loss: 0.6265 - accuracy: 0.9683 - top_5_accuracy: 0.9982 - val_loss: 1.4967 - val_accuracy: 0.7300 - val_top_5_accuracy: 0.9009 - lr: 6.5454e-04
Epoch 15/45
947/947 [==============================] - ETA: 0s - loss: 0.5805 - accuracy: 0.9812 - top_5_accuracy: 0.9992
Epoch 15: val_accuracy improved from 0.73003 to 0.73439, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 893s 943ms/step - loss: 0.5805 - accuracy: 0.9812 - top_5_accuracy: 0.9992 - val_loss: 1.4715 - val_accuracy: 0.7344 - val_top_5_accuracy: 0.9010 - lr: 5.7826e-04
Epoch 16/45
947/947 [==============================] - ETA: 0s - loss: 0.5473 - accuracy: 0.9892 - top_5_accuracy: 0.9997
Epoch 16: val_accuracy improved from 0.73439 to 0.73446, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 894s 944ms/step - loss: 0.5473 - accuracy: 0.9892 - top_5_accuracy: 0.9997 - val_loss: 1.4564 - val_accuracy: 0.7345 - val_top_5_accuracy: 0.9029 - lr: 5.0005e-04
Epoch 17/45
947/947 [==============================] - ETA: 0s - loss: 0.5223 - accuracy: 0.9938 - top_5_accuracy: 0.9999
Epoch 17: val_accuracy improved from 0.73446 to 0.74462, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 894s 944ms/step - loss: 0.5223 - accuracy: 0.9938 - top_5_accuracy: 0.9999 - val_loss: 1.4229 - val_accuracy: 0.7446 - val_top_5_accuracy: 0.9044 - lr: 4.2184e-04
Epoch 18/45
947/947 [==============================] - ETA: 0s - loss: 0.5034 - accuracy: 0.9969 - top_5_accuracy: 0.9999
Epoch 18: val_accuracy improved from 0.74462 to 0.75215, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 894s 944ms/step - loss: 0.5034 - accuracy: 0.9969 - top_5_accuracy: 0.9999 - val_loss: 1.3981 - val_accuracy: 0.7521 - val_top_5_accuracy: 0.9081 - lr: 3.4556e-04
Epoch 19/45
947/947 [==============================] - ETA: 0s - loss: 0.4896 - accuracy: 0.9983 - top_5_accuracy: 1.0000
Epoch 19: val_accuracy improved from 0.75215 to 0.75432, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 895s 945ms/step - loss: 0.4896 - accuracy: 0.9983 - top_5_accuracy: 1.0000 - val_loss: 1.3898 - val_accuracy: 0.7543 - val_top_5_accuracy: 0.9082 - lr: 2.7308e-04
Epoch 20/45
947/947 [==============================] - ETA: 0s - loss: 0.4795 - accuracy: 0.9991 - top_5_accuracy: 1.0000
Epoch 20: val_accuracy improved from 0.75432 to 0.75644, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 894s 944ms/step - loss: 0.4795 - accuracy: 0.9991 - top_5_accuracy: 1.0000 - val_loss: 1.3829 - val_accuracy: 0.7564 - val_top_5_accuracy: 0.9081 - lr: 2.0619e-04
Epoch 21/45
947/947 [==============================] - ETA: 0s - loss: 0.4719 - accuracy: 0.9994 - top_5_accuracy: 1.0000
Epoch 21: val_accuracy improved from 0.75644 to 0.75795, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 895s 945ms/step - loss: 0.4719 - accuracy: 0.9994 - top_5_accuracy: 1.0000 - val_loss: 1.3786 - val_accuracy: 0.7580 - val_top_5_accuracy: 0.9087 - lr: 1.4653e-04
Epoch 22/45
947/947 [==============================] - ETA: 0s - loss: 0.4666 - accuracy: 0.9998 - top_5_accuracy: 1.0000
Epoch 22: val_accuracy improved from 0.75795 to 0.76000, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3.keras
947/947 [==============================] - 895s 945ms/step - loss: 0.4666 - accuracy: 0.9998 - top_5_accuracy: 1.0000 - val_loss: 1.3731 - val_accuracy: 0.7600 - val_top_5_accuracy: 0.9104 - lr: 9.5582e-05
Epoch 23/45
947/947 [==============================] - ETA: 0s - loss: 0.4629 - accuracy: 0.9997 - top_5_accuracy: 1.0000
Epoch 23: val_accuracy did not improve from 0.76000
947/947 [==============================] - 893s 943ms/step - loss: 0.4629 - accuracy: 0.9997 - top_5_accuracy: 1.0000 - val_loss: 1.3743 - val_accuracy: 0.7595 - val_top_5_accuracy: 0.9110 - lr: 5.4591e-05
Epoch 24/45
947/947 [==============================] - ETA: 0s - loss: 0.4606 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 24: val_accuracy did not improve from 0.76000
947/947 [==============================] - 896s 947ms/step - loss: 0.4606 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.3759 - val_accuracy: 0.7583 - val_top_5_accuracy: 0.9107 - lr: 2.4569e-05
Epoch 25/45
947/947 [==============================] - ETA: 0s - loss: 0.4598 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 25: val_accuracy did not improve from 0.76000
947/947 [==============================] - 919s 970ms/step - loss: 0.4598 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.3758 - val_accuracy: 0.7591 - val_top_5_accuracy: 0.9102 - lr: 6.2552e-06
Epoch 26/45
947/947 [==============================] - ETA: 0s - loss: 1.2159 - accuracy: 0.7884 - top_5_accuracy: 0.9415
Epoch 26: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 1.2159 - accuracy: 0.7884 - top_5_accuracy: 0.9415 - val_loss: 2.3917 - val_accuracy: 0.5283 - val_top_5_accuracy: 0.7555 - lr: 0.0010
Epoch 27/45
947/947 [==============================] - ETA: 0s - loss: 0.7808 - accuracy: 0.9094 - top_5_accuracy: 0.9879
Epoch 27: val_accuracy did not improve from 0.76000
947/947 [==============================] - 955s 1s/step - loss: 0.7808 - accuracy: 0.9094 - top_5_accuracy: 0.9879 - val_loss: 1.7057 - val_accuracy: 0.6821 - val_top_5_accuracy: 0.8672 - lr: 9.9384e-04
Epoch 28/45
947/947 [==============================] - ETA: 0s - loss: 0.6576 - accuracy: 0.9481 - top_5_accuracy: 0.9960
Epoch 28: val_accuracy did not improve from 0.76000
947/947 [==============================] - 951s 1s/step - loss: 0.6576 - accuracy: 0.9481 - top_5_accuracy: 0.9960 - val_loss: 1.6047 - val_accuracy: 0.7009 - val_top_5_accuracy: 0.8818 - lr: 9.7553e-04
Epoch 29/45
947/947 [==============================] - ETA: 0s - loss: 0.6023 - accuracy: 0.9655 - top_5_accuracy: 0.9980
Epoch 29: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.6023 - accuracy: 0.9655 - top_5_accuracy: 0.9980 - val_loss: 1.5878 - val_accuracy: 0.7140 - val_top_5_accuracy: 0.8873 - lr: 9.4551e-04
Epoch 30/45
947/947 [==============================] - ETA: 0s - loss: 0.5856 - accuracy: 0.9693 - top_5_accuracy: 0.9988
Epoch 30: val_accuracy did not improve from 0.76000
947/947 [==============================] - 951s 1s/step - loss: 0.5856 - accuracy: 0.9693 - top_5_accuracy: 0.9988 - val_loss: 1.6046 - val_accuracy: 0.7136 - val_top_5_accuracy: 0.8848 - lr: 9.0452e-04
Epoch 31/45
947/947 [==============================] - ETA: 0s - loss: 0.5648 - accuracy: 0.9751 - top_5_accuracy: 0.9991
Epoch 31: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.5648 - accuracy: 0.9751 - top_5_accuracy: 0.9991 - val_loss: 1.5903 - val_accuracy: 0.7203 - val_top_5_accuracy: 0.8867 - lr: 8.5357e-04
Epoch 32/45
947/947 [==============================] - ETA: 0s - loss: 0.5362 - accuracy: 0.9828 - top_5_accuracy: 0.9993
Epoch 32: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.5362 - accuracy: 0.9828 - top_5_accuracy: 0.9993 - val_loss: 1.5621 - val_accuracy: 0.7219 - val_top_5_accuracy: 0.8914 - lr: 7.9391e-04
Epoch 33/45
947/947 [==============================] - ETA: 0s - loss: 0.5154 - accuracy: 0.9883 - top_5_accuracy: 0.9998
Epoch 33: val_accuracy did not improve from 0.76000
947/947 [==============================] - 952s 1s/step - loss: 0.5154 - accuracy: 0.9883 - top_5_accuracy: 0.9998 - val_loss: 1.5464 - val_accuracy: 0.7308 - val_top_5_accuracy: 0.8892 - lr: 7.2702e-04
Epoch 34/45
947/947 [==============================] - ETA: 0s - loss: 0.5058 - accuracy: 0.9897 - top_5_accuracy: 0.9999
Epoch 34: val_accuracy did not improve from 0.76000
947/947 [==============================] - 952s 1s/step - loss: 0.5058 - accuracy: 0.9897 - top_5_accuracy: 0.9999 - val_loss: 1.5230 - val_accuracy: 0.7326 - val_top_5_accuracy: 0.8933 - lr: 6.5454e-04
Epoch 35/45
947/947 [==============================] - ETA: 0s - loss: 0.4885 - accuracy: 0.9936 - top_5_accuracy: 1.0000
Epoch 35: val_accuracy did not improve from 0.76000
947/947 [==============================] - 958s 1s/step - loss: 0.4885 - accuracy: 0.9936 - top_5_accuracy: 1.0000 - val_loss: 1.5278 - val_accuracy: 0.7338 - val_top_5_accuracy: 0.8931 - lr: 5.7826e-04
Epoch 36/45
947/947 [==============================] - ETA: 0s - loss: 0.4742 - accuracy: 0.9966 - top_5_accuracy: 1.0000
Epoch 36: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.4742 - accuracy: 0.9966 - top_5_accuracy: 1.0000 - val_loss: 1.4979 - val_accuracy: 0.7392 - val_top_5_accuracy: 0.8958 - lr: 5.0005e-04
Epoch 37/45
947/947 [==============================] - ETA: 0s - loss: 0.4673 - accuracy: 0.9974 - top_5_accuracy: 1.0000
Epoch 37: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.4673 - accuracy: 0.9974 - top_5_accuracy: 1.0000 - val_loss: 1.4771 - val_accuracy: 0.7427 - val_top_5_accuracy: 0.8991 - lr: 4.2184e-04
Epoch 38/45
947/947 [==============================] - ETA: 0s - loss: 0.4588 - accuracy: 0.9985 - top_5_accuracy: 1.0000
Epoch 38: val_accuracy did not improve from 0.76000
947/947 [==============================] - 952s 1s/step - loss: 0.4588 - accuracy: 0.9985 - top_5_accuracy: 1.0000 - val_loss: 1.4704 - val_accuracy: 0.7496 - val_top_5_accuracy: 0.8995 - lr: 3.4556e-04
Epoch 39/45
947/947 [==============================] - ETA: 0s - loss: 0.4521 - accuracy: 0.9994 - top_5_accuracy: 1.0000
Epoch 39: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.4521 - accuracy: 0.9994 - top_5_accuracy: 1.0000 - val_loss: 1.4584 - val_accuracy: 0.7504 - val_top_5_accuracy: 0.9015 - lr: 2.7308e-04
Epoch 40/45
947/947 [==============================] - ETA: 0s - loss: 0.4483 - accuracy: 0.9997 - top_5_accuracy: 1.0000
Epoch 40: val_accuracy did not improve from 0.76000
947/947 [==============================] - 952s 1s/step - loss: 0.4483 - accuracy: 0.9997 - top_5_accuracy: 1.0000 - val_loss: 1.4550 - val_accuracy: 0.7518 - val_top_5_accuracy: 0.9028 - lr: 2.0619e-04
Epoch 41/45
947/947 [==============================] - ETA: 0s - loss: 0.4455 - accuracy: 0.9998 - top_5_accuracy: 1.0000
Epoch 41: val_accuracy did not improve from 0.76000
947/947 [==============================] - 952s 1s/step - loss: 0.4455 - accuracy: 0.9998 - top_5_accuracy: 1.0000 - val_loss: 1.4504 - val_accuracy: 0.7535 - val_top_5_accuracy: 0.9020 - lr: 1.4653e-04
Epoch 42/45
947/947 [==============================] - ETA: 0s - loss: 0.4437 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 42: val_accuracy did not improve from 0.76000
947/947 [==============================] - 954s 1s/step - loss: 0.4437 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.4556 - val_accuracy: 0.7522 - val_top_5_accuracy: 0.9020 - lr: 9.5582e-05
Epoch 43/45
947/947 [==============================] - ETA: 0s - loss: 0.4421 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 43: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.4421 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.4470 - val_accuracy: 0.7556 - val_top_5_accuracy: 0.9034 - lr: 5.4591e-05
Epoch 44/45
947/947 [==============================] - ETA: 0s - loss: 0.4414 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 44: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.4414 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.4450 - val_accuracy: 0.7560 - val_top_5_accuracy: 0.9022 - lr: 2.4569e-05
Epoch 45/45
947/947 [==============================] - ETA: 0s - loss: 0.4408 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 45: val_accuracy did not improve from 0.76000
947/947 [==============================] - 953s 1s/step - loss: 0.4408 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.4449 - val_accuracy: 0.7560 - val_top_5_accuracy: 0.9028 - lr: 6.2552e-06

In [19]:
results = model.evaluate(test_ds)

395/395 [==============================] - 234s 591ms/step - loss: 1.2266 - accuracy: 0.8025 - top_5_accuracy: 0.9337
